<a href="https://colab.research.google.com/github/hendrikyong/CVNL_Assignment_1/blob/main/CVNL_P02_GP01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing dataset from kaggle
import kagglehub
import os

#download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

#list files in the dataset folder
print("Path to dataset files:", path)
files = os.listdir(path)
print("Files in the dataset:", files)

#define paths to the training and testing data directories
train_dir = '/root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train'
test_dir = '/root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_test/asl_alphabet_test'

#reorganise files
#what is needed to be done is that from the test_dir for each image get first letter since all images are
#labelled as letter_test.jpg and create a folder and add that image inside that folder
import shutil

for filename in os.listdir(test_dir):
    if filename.endswith('.jpg'): #checks that it is a jpg
        letter = filename.split('_')[0] #get the first part of the filename before '_'

        #create a folder for the letter if it doesn't exist
        letter_folder = os.path.join(test_dir, letter)
        if not os.path.exists(letter_folder):
            os.makedirs(letter_folder)
        shutil.move(os.path.join(test_dir, filename), os.path.join(letter_folder, filename)) #moves the image into the corresponding folder

print("Data re-organising complete")

100%|██████████| 1.03G/1.03G [00:15<00:00, 70.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1
Files in the dataset: ['asl_alphabet_test', 'asl_alphabet_train']
Data re-organising complete


In [2]:
#imports
import os
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
from tqdm import tqdm

In [3]:
#to calculate mean and std
def calc_mean_std():
  pass

In [4]:
#model cnn model
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
'''
typical architecture of a CNN
1. input
2. conv
3. relu
4. pooling
5. fully connected layers
5. output pred

considerations:
how many conv layers do i need for feature extraction?
how many hidden layers?
how many channels?
'''

class CNN(nn.Module):
    def __init__(self, num_classes=29):
        super(CNN, self).__init__()

        self.model = nn.Sequential(
            #conv1
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #conv2
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #conv3
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),  #flatten the tensor for fully connected layers
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  #dropout for regularization
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x

In [5]:
#model and train the CNN
'''
1. input data
2. preprocessing
  - standardize images(dont need to images from the dataset already comes 200x200)
  - color transformation
  - rotate image
3. feature extraction
4. ML model
'''

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

#normalization
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomCrop(128, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) #need to change the mean and std but im not sure change to what though
    #i think need to calc mean and std with like the train_loader len(dataset) thingy ?
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)


test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomCrop(128, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
#check if gpu available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(device)
else:
    device = torch.device("cpu")
    print(device)


epochs = 5
lr = 0.001
model = CNN(num_classes=29)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
model.to(device)

cuda


CNN(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32768, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=29, bias=True)
  )
)

In [7]:
def train(model, train_loader, loss_func, optimizer, device, epochs):
    model.train()

    for epoch in range(epochs):
        epoch_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            #update tqdm with current metrics
            progress_bar.set_postfix({
                "Loss": f"{epoch_loss / (batch_idx + 1):.4f}",
                "Acc": f"{100. * correct / total:.2f}%"
            })

        print(f"Epoch {epoch+1}: Loss: {epoch_loss / len(train_loader):.4f}, Accuracy: {100. * correct / total:.2f}%")

In [8]:
def evaluate(model, test_loader, loss_func, device):
    model.eval()  #set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(test_loader, desc="Testing")

    with torch.no_grad():
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            #update tqdm with current metrics
            progress_bar.set_postfix({
                "Loss": f"{test_loss / (total):.4f}",
                "Acc": f"{100. * correct / total:.2f}%"
            })

    print(f"Test Loss: {test_loss / len(test_loader):.4f}, Accuracy: {100. * correct / total:.2f}%")

In [9]:
train(model, train_loader, loss_func, optimizer, device, epochs)
evaluate(model, test_loader, loss_func, device)

Epoch 1/5: 100%|██████████| 1360/1360 [03:54<00:00,  5.79it/s, Loss=2.1809, Acc=31.96%]


Epoch 1: Loss: 2.1809, Accuracy: 31.96%


Epoch 2/5: 100%|██████████| 1360/1360 [03:55<00:00,  5.79it/s, Loss=1.0831, Acc=61.79%]


Epoch 2: Loss: 1.0831, Accuracy: 61.79%


Epoch 3/5: 100%|██████████| 1360/1360 [03:57<00:00,  5.73it/s, Loss=0.8137, Acc=71.00%]


...

Epoch 4: Loss: 0.6710, Accuracy: 76.15%


Epoch 5/5: 100%|██████████| 1360/1360 [03:53<00:00,  5.82it/s, Loss=0.5821, Acc=79.23%]


Epoch 5: Loss: 0.5821, Accuracy: 79.23%


Testing: 100%|██████████| 1/1 [00:00<00:00,  2.62it/s, Loss=0.0635, Acc=75.00%]

Test Loss: 1.7778, Accuracy: 75.00%
